# TP Final - Análisis de Datos (CEIA - FIUBA)

**Integrantes:**
- Carolina Perez Omodeo
- Federico Arias Suárez
- Emiliano Uriel Martino
- Diego José Araujo Arellano

In [ ]:
# ---------------- Paquetes a utilizar ----------------

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import folium # Para dibujar mapas
from folium import plugins
from IPython.display import display

In [ ]:
# ---------------- Carga de datos ----------------
rent_data = pd.read_csv("listings_full.csv")

## Parte 2 - Análisis exploratorio inicial

In [ ]:
# ---------------- Visualización de las primeras filas ---------------- 

pd.options.display.max_columns = None
pd.options.display.max_rows = None
rent_data.head(5,)

In [ ]:
rent_data.shape

In [ ]:
# ---------------- Transformación de variables ---------------- 

# 'price' parece ser un objeto, por lo que habría que convertir en float
rent_data['price'] = rent_data['price'].str.replace('$','',regex=True).str.replace(',','',regex=True).astype('float')

In [ ]:
# ---------------- Resumen de algunos números ---------------- 

rent_data.loc[:,['bedrooms','beds','bathrooms','review_scores_value','price']].describe()

In [ ]:
# ---------------- Tipos de datos ---------------- 
rent_data.info()

El dataset contiene muchos campos (unos 74 en total), donde predominan las variables numéricas (41 vars.) y en menor medida variables del tipo *object* (no necesariamente categóricas).

**Variables numéricas**

Se presentan variables numéricas, tanto en valores enteros (*"int64"*), como en representaciones de punto flotante de doble precisión (*"float64"*). Algunos ejemplos de esto: 
- ***int64:*** se utilizan por lo general para hacer conteos e.g. *host_id, minimum_nights, number_of_reviews*.
- ***float64:*** se utilizan cuando se necesita representar números con varios decimales, como puede en el caso de coordenadas, e.g. *latitude, longitude, review_scores_values*.

**Variables categóricas**

Algunas variables categóricas que se pueden encontrar dentro del dataset (de tipo *"object"*), son: 
- **neighbourhood_cleansed:** que refiere al barrio donde está ubicado el alojamiento. Presenta unas 48 categorías, por lo quizás sería bueno poder agruparlos en *macro categorías* (regiones, municipios, otros) para simplificar el análisis.
- **property_type:** el tipo de propiedad (*casa, hotel, villa, etc.*). Presenta unas 78 categorías, por lo que de utilizarla sería interesante agruparlas y también presenta datos *sucios* (e.g. denominaciones en inglés y en español).
- **room_type:** que refiere a opciones en cuanto tipo de alojamiento (*'Entire home/apt', 'Private room', 'Hotel room', 'Shared room'*). 

**Variables compuestas (tipo "object")**

Como se decía anteriormente, predomina el tipo de dato *object*, que, si bien se utiliza por lo general para cadenas de texto (*strings*), puede contener cualquier tipo de dato. Algunos datos que se presentan como object, pero que podrían referir a un tipo específico de dato son:

- **Fechas:** e.g. última actualización de los datos (*last_scraped*), última puntuación (*'last_review'*), desde cuando está el host en la plataforma (*'host_since'*).
- **Lógicos:** o booleanos (`True` o `False`), e.g. la etiqueta de si el host es *superhost* (*'host_is_superhost'*), si tiene disponibilidad (*'has_availability'*).
- **Listas:** e.g. listas que indican las facilidades (*amenities*), o las verificaciones del host (*'host_verifications'*).




In [ ]:
# Histograma para ver como están distribuidos los precios de los alquileres
# En este caso estaría mejor utilizar el log de los precios, dado que tienen valores muy bajos / altos, por lo que se agrega al dataset una variable más (log_price), 
# para utilizar más adelante cualquier cosa

rent_data['log_price'] = np.log(rent_data['price'])

plt.figure(figsize=(10, 6))
sns.histplot(x=rent_data['log_price'], bins=100)
plt.title('Histograma de Precios')
plt.xlabel('Precio')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# Donde están aproximadamente los valores más frecuentes (en el entorno de los $36K)
np.exp(10.5)

## Visualización de mapa

In [ ]:
from ipyleaflet import Map, Marker, Popup
from ipywidgets import HTML

In [ ]:
# Mapa centrado de los datos (a través de la mediana)
center_map = [rent_data['latitude'].median(), rent_data['longitude'].median()]
rent_data_map = folium.Map(location=center_map, zoom_start=12)

# Colormap para los precios 
norm = mpl.colors.Normalize(vmin=rent_data['log_price'].min(), vmax=rent_data['log_price'].max())
cmap = mpl.cm.ScalarMappable(norm=norm, cmap='YlOrRd')  

# Función para escalar los valores de 'price' a radios y asignar tamaño de acuerdo al precio
def scale_radius(price, min_price, max_price, min_radius, max_radius):
    scaled_radius = ((price - min_price) / (max_price - min_price)) * (max_radius - min_radius) + min_radius
    return scaled_radius

# Valores mínimos y máximos de precio
min_price = rent_data['price'].min()
max_price = rent_data['price'].max()

# Valores mínimos y máximos del radio
min_radius = 5
max_radius = 20

# Función para añadir puntos al mapa
def add_circle_marker(row):
    color = mpl.colors.to_hex(cmap.to_rgba(row['log_price']))
    radius = scale_radius(row['log_price'], min_price, max_price, min_radius, max_radius)
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=radius,  # Radio del círculo
        color='black',
        weight=1,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=folium.Popup(f"<b>{row['name']} <br> Rating: {row['review_scores_rating']} | Reviews ({row['number_of_reviews']}) </b> <br> "
                           f"Barrio: {row['neighbourhood_cleansed']} <br> Host: {row['host_name']}  <br> Precio: ${row['price']}",
                           min_width=150, max_width=200),
        tooltip=row['id']
    ).add_to(rent_data_map)

# Se aplica la función a cada fila del DataFrame
rent_data.apply(lambda row: add_circle_marker(row), axis=1)

# Plugins - Minimapa
minimap = plugins.MiniMap()
rent_data_map.add_child(minimap)
rent_data_map

# Visualización del mapa
rent_data_map

En el mapa anterior no se pueden identificar claramente las zonas más baratas/caras para alquilar. Quizás lo mejor es segmentar por barrio y tomando una medida de centralización de los precios como puede ser la mediana (más robusta).

In [ ]:
# Mapa centrado de los datos (a través de la mediana)
center_map = [rent_data['latitude'].median(), rent_data['longitude'].median()]
rent_data_map = folium.Map(location=center_map, zoom_start=11)

# Función para calcular la mediana de precios por barrio
def medianprice_neighbourhood(df):
    median_prices = df.groupby('neighbourhood_cleansed')['price'].median().reset_index()
    median_prices = median_prices.rename(columns={'price': 'median_price'})
    return df.merge(median_prices, on='neighbourhood_cleansed')

# Aplicar la función a los datos
rent_data = medianprice_neighbourhood(rent_data)

# Colormap para los precios 
norm = mpl.colors.Normalize(vmin=rent_data['median_price'].min(), vmax=rent_data['median_price'].max())
cmap = mpl.cm.ScalarMappable(norm=norm, cmap='YlOrRd')  

# Función para escalar los valores de 'price' a radios y asignar tamaño de acuerdo al precio
def scale_radius(price, min_price, max_price, min_radius, max_radius):
    scaled_radius = ((price - min_price) / (max_price - min_price)) * (max_radius - min_radius) + min_radius
    return scaled_radius

# Valores mínimos y máximos de precio
min_price = rent_data['price'].min()
max_price = rent_data['price'].max()

# Valores mínimos y máximos del radio
min_radius = 5
max_radius = 20

# Función para añadir puntos al mapa
def add_circle_marker(row):
    color = mpl.colors.to_hex(cmap.to_rgba(row['median_price']))
    radius = scale_radius(row['log_price'], min_price, max_price, min_radius, max_radius)
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=radius,  # Radio del círculo
        color='black',
        weight=1,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=folium.Popup(f"<b>{row['name']} <br> Rating: {row['review_scores_rating']} | Reviews ({row['number_of_reviews']}) </b> <br> "
                           f"Barrio: {row['neighbourhood_cleansed']} <br> Host: {row['host_name']}  <br> Precio: ${row['price']}",
                           min_width=150, max_width=200),
        tooltip=row['id']
    ).add_to(rent_data_map)

# Se aplica la función a cada fila del DataFrame
rent_data.apply(lambda row: add_circle_marker(row), axis=1)

# Plugins - Minimapa
minimap = plugins.MiniMap()
rent_data_map.add_child(minimap)
rent_data_map

# Visualización del mapa
rent_data_map



Ahora si se pueden detectar zonas de renta donde el precio parece ser más elevado, siguiendo el patrón que cuanto más cercano a la costa, más aumenta la renta temporal de estas propiedades.